# An Attempt at DSPy Agents

If you would rather *read* this, you can find it on [LearnByBuilding.AI](https://learnbybuilding.ai/tutorials/). This notebook only contains code, to get some prose along with it, check out the tutorial posted there.

If you like this content, [follow me on twitter](https://twitter.com/bllchmbrs) for more! I'm posting all week about DSPy and providing a lot of "hard earned" lessons that I've gotten from learning the material.

In [137]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
from dotenv import load_dotenv
load_dotenv()

True

In [139]:
import dspy
import instructor

wrkr = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1000, api_base='http://0.0.0.0:4000', model_type="chat")
bss = dspy.OpenAI(model='gpt-4-turbo', max_tokens=1000, api_base='http://0.0.0.0:4000', model_type="chat")

dspy.configure(lm=wrkr)

In [140]:
from typing import List, Any, Callable, Optional
from pydantic import BaseModel

In [142]:
class Plan(dspy.Signature):
    """Produce a step by step plan to perform the task. 
The plan needs to be in markdown format and should be broken down into big steps (with ## headings) and sub-steps beneath those.
When thinking about your plan, be sure to think about the tools at your disposal and include them in your plan.
    """
    task = dspy.InputField(prefix="Task", desc="The task")
    context = dspy.InputField(format=str, desc="The context around the plan")
    proposed_plan = dspy.OutputField(desc="The proposed, step by step execution plan.")

In [163]:
class Worker(dspy.Module):
    def __init__(self, role:str, tools:List):
        self.role = role
        self.tools = tools
        self.tool_descriptions = "\n".join([f"- {t.name}: {t.description}. To use this tool please provide: `{t.requires}`" for t in tools])
        self.plan = dspy.ChainOfThought(Plan)
        self.execute_step = None
        self.history = []

    def forward(self, task:str):
        context = f"{self.role}\n{self.tool_descriptions}"
        input_args = dict(
            context = context,
            task = task
        )
        result = self.plan(**input_args)
        print(result.proposed_plan)

In [164]:
class Tool(BaseModel):
    name: str
    description: str
    requires: str
    func: Callable
    
test_tools = [
    Tool(name="phone", description="a way of making phone calls", requires="phone_number", func=lambda x:  "they've got time"),
    Tool(name="local business lookup", description="Look up businesses by category", requires="business category", func=lambda x:  "Bills landscaping: 415-555-5555")
]
with dspy.context(lm=wrkr):
    Worker("assistant", test_tools).forward("get this yard cleaned up.")

## Step 1: Assess the yard
- Use the phone to call a landscaping company to get a quote for yard cleanup services.
- Use the local business lookup to find a reputable landscaping company in the area.

## Step 2: Create a plan
- Once you have received quotes from different companies, compare prices and services offered.
- Decide on a budget and timeline for the yard cleanup.

## Step 3: Remove debris
- Start by removing any large debris such as fallen branches, leaves, and trash from the yard.
- Use a rake or leaf blower to gather smaller debris into piles for easier disposal.

## Step 4: Mow the lawn
- Use a lawn mower to cut the grass to an appropriate length.
- Trim the edges of the lawn for a clean and polished look.

## Step 5: Trim bushes and hedges
- Use pruning shears or a hedge trimmer to shape and trim bushes and hedges in the yard.
- Remove any dead or overgrown branches to promote healthy growth.

## Step 6: Weed the garden beds
- Use a garden hoe or hand tools to remove wee

In [175]:
class Worker2(dspy.Module):
    def __init__(self, role:str, tools:List):
        self.role = role
        self.tools = dict([(t.name, t) for t in tools])
        self.tool_descriptions = "\n".join([f"- {t.name}: {t.description}. To use this tool please provide: `{t.requires}`" for t in tools])
        self._plan = dspy.ChainOfThought(Plan)
        self._tool = dspy.ChainOfThought("task, context -> tool_name, tool_argument")
        self.execute_step = None
        self.history = []
        print(self.tool_descriptions)

    def plan(self, task:str, feedback:Optional[str]=None):
        context = f"Your role:{self.role}\n Tools at your disposal:\n{self.tool_descriptions}"
        if feedback:
            context += f"\nPrevious feedback on your prior plan {feedback}"
        input_args = dict(
            task=task,
            context=context
        )    
        result = self._plan(**input_args)
        return result.proposed_plan

    def execute(self, task:str):
        print(f"executing {task}")
        res = self._tool(task=task, context=self.tool_descriptions)
        t = res.tool_name
        arg = res.tool_argument
        if t in self.tools:
            complete = self.tools[t].func(arg)
        else:
            return f"{t} with {arg} completed successfully"
        

In [176]:
email_tool = Tool(
    name="email",
    description="Send and receive emails",
    requires="email_address",
    func=lambda x: f"Email sent to {x}"
)

schedule_meeting_tool = Tool(
    name="schedule meeting",
    description="Schedule meetings",
    requires="meeting_details",
    func=lambda x: f"Meeting scheduled on {x}"
)

# Tools for the janitor
cleaning_supplies_tool = Tool(
    name="cleaning supplies",
    description="List of cleaning supplies needed",
    requires="cleaning_area",
    func=lambda x: f"Need supplies for {x}"
)

maintenance_report_tool = Tool(
    name="maintenance report",
    description="Report maintenance issues",
    requires="issue_description",
    func=lambda x: f"There's too much work for one person. I need help!"
)

# Tools for the software engineer
code_compiler_tool = Tool(
    name="code compiler",
    description="Compile code",
    requires="source_code",
    func=lambda x: "Code compiled successfully"
)

bug_tracker_tool = Tool(
    name="bug tracker",
    description="Track and report bugs",
    requires="bug_details",
    func=lambda x: f"Bug reported: {x}"
)

# Tools for the cook
recipe_lookup_tool = Tool(
    name="recipe lookup",
    description="Look up recipes",
    requires="dish_name",
    func=lambda x: f"Recipe for {x} found"
)

kitchen_inventory_tool = Tool(
    name="kitchen inventory",
    description="Check kitchen inventory",
    requires="ingredient",
    func=lambda x: f"Inventory checked for {x}"
)

# Assign tools to workers
workers = [
    Worker2("assistant", [email_tool, schedule_meeting_tool]),
    Worker2("janitor", [cleaning_supplies_tool, maintenance_report_tool]),
    Worker2("software engineer", [code_compiler_tool, bug_tracker_tool]),
    Worker2("cook", [recipe_lookup_tool, kitchen_inventory_tool])
]

- email: Send and receive emails. To use this tool please provide: `email_address`
- schedule meeting: Schedule meetings. To use this tool please provide: `meeting_details`
- cleaning supplies: List of cleaning supplies needed. To use this tool please provide: `cleaning_area`
- maintenance report: Report maintenance issues. To use this tool please provide: `issue_description`
- code compiler: Compile code. To use this tool please provide: `source_code`
- bug tracker: Track and report bugs. To use this tool please provide: `bug_details`
- recipe lookup: Look up recipes. To use this tool please provide: `dish_name`
- kitchen inventory: Check kitchen inventory. To use this tool please provide: `ingredient`


In [177]:
class SubTask(BaseModel):
    action:str
    assignee: str
    
class Task(BaseModel):
    sub_tasks:List[SubTask]
    
class ParsedPlan(BaseModel):
    tasks: List[Task]

In [178]:
import instructor
from openai import OpenAI
_client = instructor.from_openai(OpenAI(base_url="http://0.0.0.0:4000/"))

def get_plan(plan:str, context:str):
    return _client.chat.completions.create(
        response_model=ParsedPlan,
        model="gpt-3.5-turbo",
        messages=[
            dict(role="system", content="You help parse markdown into a structured format."),
            dict(role="user", content=f"Here is the context about the plan: \n{context} \n\n The plan: \n\n {plan}")
        ],
    )

In [179]:
class Boss(dspy.Module):
    def __init__(self, base_context:str, direct_reports=List, lm=bss):
        self.base_context = base_context
        self._plan = dspy.ChainOfThought("task, context -> assignee")
        self._approve = dspy.ChainOfThought("task, context -> approve")
        self._critique = dspy.ChainOfThought("task, context -> critique")
        self.reports = dict((d.role,d) for d in direct_reports)
        self.lm = lm
        report_capabilities = []
        for r in direct_reports:
            report_capabilities.append(f"{r.role} has the follow tools:\n{r.tool_descriptions}")

        self.report_capabilities = "\n".join(report_capabilities) 

        print(self.report_capabilities)
        
    def critique(self, task:str, plan:str, extra_context:Optional[str]=None):
        context=self.base_context
        if extra_context:
            context += "\n"
            context += extra_context
        
        crit_args = dict(
            context=context,
            task=task,
            proposed_plan=plan)
        with dspy.context(lm=self.lm):
            result = self._critique(**crit_args)
        return result.critique

    def approve(self, task:str, plan:str, extra_context:Optional[str]=None):
        context=self.base_context + "\n You only approve plans after 2 iterations"
        if extra_context:
            context += "\n"
            context += extra_context
        
        approval_args = dict(
            context=context,
            task=task,
            proposed_plan=plan)

        with dspy.context(lm=self.lm):
            result = self._approve(**approval_args)

        return result.approve        
        

    def plan(self, task:str):
        context=self.base_context + f"Here are your direct report capabilities: {self.report_capabilities}"
        
        plan_args = dict(
            context = context,
            task=f"Which person should take on the following task: {task}"
        )
        assignee = self._plan(**plan_args).assignee
        is_assigned = assignee.lower() in self.reports
        report = None

        print("assigning")
        for x in range(3):
            if is_assigned:
                report = self.reports[assignee]
            else:
                context += f"\n\n you tried to assign to {assignee} but that's not a valid one. Think carefully and assign the proper report"
                plan_args = dict(
                    context = context,
                    task=f"Which person should take on the following task: {task}"
                )
                assignee = self._plan(**plan_args).assignee

        assert report, "Failed to assign"
        print("assigning complete")
    
        print("planning")
        reports_plan = report.plan(task)
        with dspy.context(lm=self.lm):
            approval = self.approve(task, reports_plan)
            is_approved = "yes" in approval.lower()
        
        for x in range(1):
            print(f"Cycle {x}: {approval}")
            if is_approved:
                break
            feedback = self.critique(task, reports_plan)
            feedback = f"Prior plan: {reports_plan}\n Boss's Feedback: {feedback}"
            # print(feedback)
            reports_plan = report.plan(task, feedback)
            complete = f"{feedback}\n\nNew plan:\n\n{reports_plan}"
            approval = self.approve(task, reports_plan)
            is_approved = "yes" in approval.lower()
            
        parsed_plan = get_plan(reports_plan, f"The assignee is: {assignee}. The rest of the team is: {self.report_capabilities}")
        for task in parsed_plan.tasks:
            for sub_task in task.sub_tasks:
                self.reports[sub_task.assignee].execute(sub_task.action)

In [180]:
b = Boss("You are a boss that manages a team of people, you're responsible for them doing well and completing the tasks you are given.", workers)

assistant has the follow tools:
- email: Send and receive emails. To use this tool please provide: `email_address`
- schedule meeting: Schedule meetings. To use this tool please provide: `meeting_details`
janitor has the follow tools:
- cleaning supplies: List of cleaning supplies needed. To use this tool please provide: `cleaning_area`
- maintenance report: Report maintenance issues. To use this tool please provide: `issue_description`
software engineer has the follow tools:
- code compiler: Compile code. To use this tool please provide: `source_code`
- bug tracker: Track and report bugs. To use this tool please provide: `bug_details`
cook has the follow tools:
- recipe lookup: Look up recipes. To use this tool please provide: `dish_name`
- kitchen inventory: Check kitchen inventory. To use this tool please provide: `ingredient`


In [181]:
b.plan("clean up the yard")

assigning
assigning complete
planning
Cycle 0: Pending on whether the task plan has undergone two thorough iterations. If yes, approve; if no, require further refinement.
executing Assess the yard
executing Gather cleaning supplies
executing Start cleaning
executing Inspect the yard
executing Report any maintenance issues
executing Dispose of cleaning supplies
